## Libraries and data download

In [ ]:
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from cleantext import clean #pip install clean-text
import re
# from sklearn.model_selection import train_test_split
from datasets import Dataset, ClassLabel, Value, Features,DatasetDict
from huggingface_hub import upload_folder



In [ ]:
# Download latest version
path = kagglehub.dataset_download("datasnaek/mbti-type")

print("Path to dataset files:", path)


In [ ]:
df = pd.read_csv("C:/Users/Tim/.cache/kagglehub/datasets/datasnaek/mbti-type/versions/1/mbti_1.csv")

df.head()
length_df = len(df)

# Target variable distribution 

In [ ]:
target_exp = pd.DataFrame(columns = ["type", "counts", "percentage"])
target_exp["type"] = df["type"].unique()
target_exp = df["type"].value_counts().reset_index()
target_exp["percentage"] = round(target_exp["count"] / target_exp["count"].sum() * 100, ndigits = 3)
for i in range(4):
    target_exp[f"axis {i+1} (I/E)"] = target_exp["type"].str[i]

target_exp.loc[[0],["approx. general pop. freq."]] = 7
target_exp.loc[[1],["approx. general pop. freq."]] = 2
target_exp.loc[[2],["approx. general pop. freq."]] = 4
target_exp.loc[[3],["approx. general pop. freq."]] = 3
target_exp.loc[[4],["approx. general pop. freq."]] = 3.5
target_exp.loc[[5],["approx. general pop. freq."]] = 7
target_exp.loc[[6],["approx. general pop. freq."]] = 5
target_exp.loc[[7],["approx. general pop. freq."]] = 7
target_exp.loc[[8],["approx. general pop. freq."]] = 3.5
target_exp.loc[[9],["approx. general pop. freq."]] = 12.5
target_exp.loc[[10],["approx. general pop. freq."]] = 3.5
target_exp.loc[[11],["approx. general pop. freq."]] = 11.5
target_exp.loc[[12],["approx. general pop. freq."]] = 4.5
target_exp.loc[[13],["approx. general pop. freq."]] = 6.5
target_exp.loc[[14],["approx. general pop. freq."]] = 11
target_exp.loc[[15],["approx. general pop. freq."]] = 10


target_exp


# Plotting Distribution

In [ ]:
plot_data = target_exp.melt(
    id_vars = "type",
    value_vars= ["percentage", "approx. general pop. freq."],
    var_name = "Group",
    value_name = "Percentage"
)

plot_data["Group"] = plot_data["Group"].replace({
    "percentage": "Sample",
    "approx. general pop. freq.": "General Population (approx.)"
})

fig, ax1 = plt.subplots(figsize=(10,6))


sns.barplot(data = plot_data, x = "type", y = "Percentage", hue = "Group", alpha = 0.6)
ax1.set_ylabel("Percentage")
ax2 = ax1.twinx()

ax2.set_ylabel("Count (Sample only)")
total_n = target_exp["count"].sum()
max_perc = max(target_exp["percentage"].max(), target_exp["approx. general pop. freq."].max())


ax1.set_ylim(0, max_perc * 1.1)  
ax2.set_ylim(0, (max_perc * 1.1) * total_n / 100)
ax1.set_xlabel("MBTI Type")
plt.legend()
plt.title("MBTI Distribution Dataset vs General Population")

# Data Cleaning and Splitting
- mask urls
- mask emails
- mask phone numbers
- mask ip addresses
- mask file paths
- mask mbti type mentions
- drop rows with less than 30 characters
- drop rows that are only numeric
- drop duplicates

## Splitting
Each observation of "posts" in the dataset represents a collection of forum posts by a user. Since these posts are usually not related and there is no "dialogue"-like structure to them, splitting them allows for generating more data points ("50-times more") without any/much loss of information.

In [ ]:
# cleaning function
def clean_data(text):
    return clean(text,
           fix_unicode = True,
           no_urls = True,
           no_emails = True,
           no_file_paths = True,
           no_phone_numbers = True,
           no_ip_addresses = True
           #no_emoji=True
           )

# splitting function
def split_strings(data, col_to_split, separator):
    dfcopy = data.copy().astype(str)
    dfcopy[col_to_split] = dfcopy[col_to_split].str.split(pat = separator, regex=False)
    return dfcopy.explode(col_to_split).reset_index(drop = True)


# replace mbti mentions
def replace_mbti(text):
    pattern = r'\b(infj|infp|intj|intp|isfj|isfp|istj|istp|enfj|enfp|entj|entp|esfj|esfp|estj|estp)\b'
    return re.sub(pattern, "<mbti>", text, flags=re.IGNORECASE)

def preprocessing_pipe(df, col_to_clean, col_cleaned, separator):
    # cleaning
    df[col_cleaned] = df[col_to_clean].apply(clean_data)

    #mbti type removal
    df[col_cleaned] = df[col_cleaned].apply(replace_mbti)

    #splitting
    df = split_strings(data = df, col_to_split= col_cleaned, separator= separator)

    #drop short posts
    df = df.loc[df[col_cleaned].str.len() > 30].reset_index(drop = True)

    # drop posts that are just numbers
    df = df[~df[col_cleaned].str.isnumeric()]

    # drop possible duplicates
    df = df.drop_duplicates(subset=[col_cleaned])
    df = df.drop(columns=[col_to_clean])
    return df


df_clean = preprocessing_pipe(df, "posts", "posts_cleaned", "|||")
length_cleaned = len(df_clean)


# Comparison cleaned vs not cleaned

In [ ]:
print(f"Length of initial dataset: {length_df} \n Length of cleaned dataset: {length_cleaned}")

In [ ]:
target_exp_clean = pd.DataFrame(columns = ["type", "counts", "percentage"])
target_exp_clean["type"] = df_clean["type"].unique()
target_exp_clean = df_clean["type"].value_counts().reset_index()
target_exp_clean["percentage"] = round(target_exp_clean["count"] / target_exp_clean["count"].sum() * 100, ndigits = 3)
for i in range(4):
    target_exp_clean[f"axis {i+1} (I/E)"] = target_exp_clean["type"].str[i]

target_exp_clean.loc[[0],["approx. general pop. freq."]] = 7
target_exp_clean.loc[[1],["approx. general pop. freq."]] = 2
target_exp_clean.loc[[2],["approx. general pop. freq."]] = 4
target_exp_clean.loc[[3],["approx. general pop. freq."]] = 3
target_exp_clean.loc[[4],["approx. general pop. freq."]] = 3.5
target_exp_clean.loc[[5],["approx. general pop. freq."]] = 7
target_exp_clean.loc[[6],["approx. general pop. freq."]] = 5
target_exp_clean.loc[[7],["approx. general pop. freq."]] = 7
target_exp_clean.loc[[8],["approx. general pop. freq."]] = 3.5
target_exp_clean.loc[[9],["approx. general pop. freq."]] = 12.5
target_exp_clean.loc[[10],["approx. general pop. freq."]] = 3.5
target_exp_clean.loc[[11],["approx. general pop. freq."]] = 11.5
target_exp_clean.loc[[12],["approx. general pop. freq."]] = 4.5
target_exp_clean.loc[[13],["approx. general pop. freq."]] = 6.5
target_exp_clean.loc[[14],["approx. general pop. freq."]] = 11
target_exp_clean.loc[[15],["approx. general pop. freq."]] = 10


target_exp_clean

In [ ]:


plot_data_clean = target_exp_clean.melt(
    id_vars = "type",
    value_vars= ["percentage", "approx. general pop. freq."],
    var_name = "Group",
    value_name = "Percentage"
)

plot_data_clean["Group"] = plot_data_clean["Group"].replace({
    "percentage": "Sample",
    "approx. general pop. freq.": "General Population (approx.)"
})

fig, ax1 = plt.subplots(figsize=(10,6))


sns.barplot(data = plot_data_clean, x = "type", y = "Percentage", hue = "Group", alpha = 0.6)
ax1.set_ylabel("Percentage")
ax2 = ax1.twinx()

ax2.set_ylabel("Count (Sample only)")
total_n = target_exp_clean["count"].sum()
max_perc = max(target_exp_clean["percentage"].max(), target_exp_clean["approx. general pop. freq."].max())


ax1.set_ylim(0, max_perc * 1.1)  
ax2.set_ylim(0, (max_perc * 1.1) * total_n / 100)
ax1.set_xlabel("MBTI Type")
plt.legend()
plt.title("MBTI Distribution in Cleaned Dataset vs General Population")


In [ ]:
# 1. Erstelle das Grid (1 Zeile, 2 Spalten)
fig, (axes_left, axes_right) = plt.subplots(2, 1, figsize=(18, 7))

# --- DEIN PLOT (kommt auf die linke Seite: axes_left) ---
# Wichtig: ax = axes_left zuweisen!
sns.barplot(data=plot_data_clean, x="type", y="Percentage", hue="Group", alpha=0.6, ax=axes_left)

axes_left.set_ylabel("Percentage")
axes_left.set_xlabel("MBTI Type")

# Jetzt das twinx basierend auf axes_left erstellen
ax2 = axes_left.twinx() 
ax2.set_ylabel("Count (Sample only)")

# Skalierung (Deine Logik übernommen)
total_n = target_exp["count"].sum()
max_perc = max(target_exp["percentage"].max(), target_exp["approx. general pop. freq."].max())

axes_left.set_ylim(0, max_perc * 1.1)
ax2.set_ylim(0, (max_perc * 1.1) * total_n / 100)

axes_left.set_title("MBTI Distribution vs General Population")

# --- DER ZWEITE PLOT (kommt auf die rechte Seite: axes_right) ---
sns.barplot(data = plot_data_clean, x = "type", y = "Percentage", hue = "Group", alpha = 0.6,  ax = axes_right)
ax1.set_ylabel("Percentage")
ax2 = ax1.twinx()

ax2.set_ylabel("Count (Sample only)")
total_n = target_exp_clean["count"].sum()
max_perc = max(target_exp_clean["percentage"].max(), target_exp_clean["approx. general pop. freq."].max())


ax1.set_ylim(0, max_perc * 1.1)  
ax2.set_ylim(0, (max_perc * 1.1) * total_n / 100)
ax1.set_xlabel("MBTI Type")
plt.legend()
plt.title("MBTI Distribution in Cleaned Dataset vs General Population")

plt.tight_layout()
plt.show()

In [ ]:
rseed = 42

df_clean = df_clean.rename(columns={"type": "label", "posts_cleaned": "post"})
df_hf = Dataset.from_pandas(df_clean, preserve_index=False)


df_hf = df_hf.class_encode_column("label")
mbti_labels = ["ENFJ", "ENFP", "ENTJ", "ENTP", "ESFJ", "ESFP", "ESTJ", "ESTP", 
               "INFJ", "INFP", "INTJ", "INTP", "ISFJ", "ISFP", "ISTJ", "ISTP"]


new_features = df_hf.features.copy()
new_features["label"] = ClassLabel(names=mbti_labels)

df_hf = df_hf.cast(new_features)

split = df_hf.train_test_split(test_size = 0.2, shuffle = True, stratify_by_column="label", seed = rseed)
train = split["train"]
df_temp = split["test"]

split2 = df_temp.train_test_split(test_size = 0.5, shuffle = True, stratify_by_column= "label", seed = rseed)
val = split2["train"]
test = split2["test"]

df_final = DatasetDict({
    "train": train,
    "test": test,
    "validation": val
})

# Save the dataset

In [ ]:
#df_clean.to_csv("..\data\mbti_cleaned_whole.csv", header = ["label", "post"], index = False)
#df_final.save_to_disk("..\data\mbti_dict_ver2")


#upload to HF
#upload_folder(folder_path="..\data\mbti_dict_ver2", repo_id="DrinkIcedT/mbti", repo_type="dataset")
# upload_folder(folder_path="..\data\smol", repo_id="DrinkIcedT/smol_mbti", repo_type="dataset")

# Notes:
- split before cleaning
- drop rows under ??? words/characters
    - drop rows that are only numbers

After cleaning is done, plot distribution again